In [1]:
import tensorflow
import gym

In [10]:
#env = gym.make('CartPole-v0')
env = gym.make('CartPole-v2')
#env = gym.make('CartPoleEnv')
env.reset()
for _ in range(1000):
    env.render()
    env.step(env.action_space.sample()) # take a random action

You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.


## Observations

Num 	Observation 	Min 	Max

0 	Cart Position 	-2.4 	2.4

1 	Cart Velocity 	-Inf 	Inf

2 	Pole Angle 	~ -41.8° 	~ 41.8°

3 	Pole Velocity At Tip 	-Inf 	Inf


## Actions
0 	Push cart to the left
1 	Push cart to the right


Source: 
https://github.com/openai/gym/wiki/CartPole-v0

In [9]:
import numpy as np
import itertools

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

#import progressbar
import operator

# Similar to sum, but with multiplication
mul = lambda seq: reduce(operator.mul, seq)


#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Global Variables
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++  

import gym
env  = gym.make('CartPole-v2')

o_space = env.observation_space
a_space = env.action_space

o_dim = o_space.shape[0]
o_high = o_space.high 
o_low = o_space.low 
o_range = zip(o_low,o_high)

o_cells = [10,10,10,10]
o_bins = [np.linspace( low, high, cells) for (low,high), cells in zip(o_range, o_cells)]

# Holds the offset which is required for calculating the linearized index from the cell indices
# The offset for each dimension is defined by the number of cells in previous dimensions
offset = [1]
for i in range(1,o_dim):
    offset.append( o_cells[i - 1] * offset[i - 1] )




#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Functions
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++  

obs_to_state_tuple = lambda obs: [ int(np.digitize(obs[i], o_bins[i])) for i in range(o_dim)]
obs_to_state = lambda obs: sum([ a * b for a,b in zip( obs_to_state_tuple(obs), offset)])

#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Code to run the stuff
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++  

#NS = mul(o_cells)
NS = np.prod(o_cells)
NA = 2

gamma = 0.9 # Discount factor
alpha = 0.1 # Constant learning rate

Q = np.zeros( (NS,NA) )
pi = np.random.randint(0,NA, size = (NS,))

for epoch in range(10000):
    print("Epoch:", epoch)
    print ("Training")
    o = env.reset()
    s = obs_to_state(o)

    # Adjust this to a suitable value for your computer
    for i in range(10):

        # IMPORTANT: This was not shown in the tutorial and is the reason why the agent did not get significantly better within the considered time.
        # Random actions are required to learn about other (s,a) tuples and not only about those, that are produced by following the policy pi
        a = pi[s] if np.random.rand() < 0.90 else a_space.sample()

        o2, r, done, _ = env.step(a)

        s2 = obs_to_state(o2)

        # Handle episodic tasks
        Q2 = Q[s2, :].max() if not done else 0.0

        # TD target
        Q_tgt = r + gamma * Q2

        # TD error
        delta = Q_tgt - Q[s, a]

        # Q-Learning
        Q[s,a] = Q[s,a] + alpha * delta

        # Next policy
        pi = Q.argmax(1)

        # Copy values for next round
        s = s2

        if done:
            o = env.reset()
            s = obs_to_state(o)
        #end if
    #end for training

    print("Running")
    for i in range(5):
        c = 0
        o = env.reset()
        done = False
    while not done:

        c += 1
        s = obs_to_state(o)
        a = pi[s] 
        o, _, done, _ = env.step(a)
        env.render()

    #end while

    print("Steps: " , c)
    #end for running
#end for epochs


Epoch: 0
Training
Running
Steps:  8
Epoch: 1
Training
Running
Steps:  8
Epoch: 2
Training
Running
Steps:  9
Epoch: 3
Training
Running
Steps:  10
Epoch: 4
Training
Running
Steps:  10
Epoch: 5
Training
Running
Steps:  8
Epoch: 6
Training
Running
Steps:  9
Epoch: 7
Training
Running
Steps:  10
Epoch: 8
Training
Running
Steps:  8
Epoch: 9
Training
Running
Steps:  10
Epoch: 10
Training
Running
Steps:  9
Epoch: 11
Training
Running
Steps:  9
Epoch: 12
Training
Running
Steps:  9
Epoch: 13
Training
Running
Steps:  10
Epoch: 14
Training
Running
Steps:  9
Epoch: 15
Training
Running
Steps:  10
Epoch: 16
Training
Running
Steps:  9
Epoch: 17
Training
Running
Steps:  9
Epoch: 18
Training
Running
Steps:  8
Epoch: 19
Training
Running
Steps:  10
Epoch: 20
Training
Running
Steps:  9
Epoch: 21
Training
Running
Steps:  9
Epoch: 22
Training
Running
Steps:  10
Epoch: 23
Training
Running
Steps:  9
Epoch: 24
Training
Running
Steps:  10
Epoch: 25
Training
Running
Steps:  8
Epoch: 26
Training
Running
Steps:  10
E

ArgumentError: argument 2: <class 'TypeError'>: wrong type